In [1]:
import numpy as np
import xobjects as xo
import xtrack as xt
import xpart as xp
import json
import pandas as pd
from cpymad.madx import Madx
#from matplotlib import pyplot as plt
import NAFFlib
from math import modf
#from scipy.constants import physical_constants
from matplotlib import pyplot as plt
from scipy.stats import linregress

EOFError: marshal data too short

In [ ]:
#--------input and context---------
i_mo = -350
p0c = 7000e9
normal_emitt_x = 3e-6 #m*rad
normal_emitt_y = 3e-6 #m*rad
N_particles = 11 
N=1000
n_turns= 1

ctx_cpu = xo.ContextCpu()

#--------Opening the file for the first time, then you can if False this section---------
mad = Madx()
mad.option(echo=False)
mad.call('lhcandrea4.madx')
mad.use(sequence="lhcb1")
line = xt.Line.from_madx_sequence(mad.sequence['lhcb1'],
                              deferred_expressions=True
                              )
with open('hl_line.json', 'w') as fid:
    json.dump(line.to_dict(), fid, cls=xo.JEncoder)

#--------This is False can be removed after the creation of the file--------------    
    
if False:
    with open('hl_line.json', 'r') as fid:
        loaded_dct = json.load(fid)
    line = xt.Line.from_dict(loaded_dct)

#-----preparing for tracking------

#particle_0 = xp.Particles(_context=ctx_cpu, p0c=p0c)
particle_0 = xp.Particles(mass0=xp.PROTON_MASS_EV, q0=1, p0c=p0c, x=1e-3, y=1e-3)
line.vars['i_mo'] = i_mo

#-----sanity checks--------

print(line.vars['i_mo']._get_value())
print(line.vars['cmrskew']._get_value())

tracker_normal = xt.Tracker(_context=ctx_cpu, line=line)
ref = tracker_normal.find_closed_orbit(particle_0)
print(ref.x,ref.y,ref.zeta)
tw_normal = tracker_normal.twiss(ref)


betx_at_ip3 = tw_normal['betx'][0]
bety_at_ip3 = tw_normal['bety'][0]
print(betx_at_ip3,bety_at_ip3)

#----defining the sigma in both directions---------

sigma_x = np.sqrt(betx_at_ip3*normal_emitt_x/(particle_0.gamma0*particle_0.beta0))
sigma_y = np.sqrt(bety_at_ip3*normal_emitt_y/(particle_0.gamma0*particle_0.beta0))
print(sigma_x,sigma_y)

#-----closed orbit particle-----

#p0_normal = tw_normal['particle_on_co']
p0_normal = ref 
print(p0_normal.x,p0_normal.y,p0_normal.zeta)

#-----choose one of the two to have a particle with x or y displacement----------------

#particles_normal = xp.build_particles(_context=ctx_cpu,particle_ref=p0_normal,
#                                     x=[mysigma*sigma_x[0] for mysigma in np.linspace(0.2,2.2,N_particles)])
particles_normal = xp.build_particles(_context=ctx_cpu,particle_ref=p0_normal,
                                      y=[mysigma*sigma_y[0] for mysigma in np.linspace(0.2,2.2,N_particles)])

#-----tracking------------------

my_result = {}
for ii in ['x','px','y','py','zeta','delta']:
    my_result[ii] = []


for ii in range(N):
    tracker_normal.track(particles_normal, num_turns=n_turns,turn_by_turn_monitor=False)
    for jj in ['x','px','y','py','zeta','delta']:
        my_result[jj].append(getattr(particles_normal,jj).copy())       
        #my_result[jj].append(getattr(particles_normal,jj).copy())

for jj in ['x','px','y','py','zeta','delta']:
        my_result[jj] = np.array(my_result[jj])
        #my_result[jj] = np.array(my_result[jj])


#-------tunes-------

qx_normal = []
qy_normal = []

for ii in range(N_particles):
    
    qx_normal.append(NAFFlib.get_tune(my_result['x'][:,ii]))
    qy_normal.append(NAFFlib.get_tune(my_result['y'][:,ii]))

x0 = np.array([mysigma*sigma_x[0] for mysigma in np.linspace(0.2,2.2,N_particles)]) 
y0 = np.array([mysigma*sigma_y[0] for mysigma in np.linspace(0.2,2.2,N_particles)]) 